#### 사용자 정의 함수 

----
- 함수기능: 모델 학습 진행 함수
- 함수이름: training
- 매개변수: 함수 구동시 필요한 재료
            - 모델 인스턴스
            - 학습데이터셋: 피쳐와 타겟(학습전 Tensor화)  
            - 손실함수 인스턴스  
            - 최적화 인스턴스  
            - 학습횟수: EPOCH  
            - 배치크기: BATH_SIZE  
            - 배치개수: BATCH_CNT, BATCH_ITER  
            - 검증용 데이터셋: 피쳐와 타겟 (검증전 Tensor화)  
- 함수결과: 학습시 오포크당 손실값과 성능지표값
----

---
- 함수기능: 모델 학습 진행 함수
- 함수이름: epochTraining
- 매개변수: 학습을 위한 재료들
            - 모델 인스턴스
            - 학습용 데이터셋(피쳐, 타겟)
            - 손실함수 인스턴스
            - 최적화 인스턴스
            - 배치크기
            - 배치개수
- 함수결과: 손실값과 성능지표값
---

In [2]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

# 전역변수 (텐서 저장 및 실행 위치)
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'


In [3]:
# 검증 및 테스트 진행 후 손실값과 성능지표 반환
def epochTesting(model, feature, target, lossFunc, scoreFunc):
    loss_total, score_total=0, 0

    # 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행
    with torch.no_grad():
        # 데이터셋 추출 및 텐서화

        X_val=torch.FloatTensor(feature.values).to(DEVICE)
        Y_val=torch.FloatTensor(target.values).reshape(-1,1).to(DEVICE)

        # 학습 진행
        pre_y=model(X_val).to(DEVICE)

        # 손실계산
        loss=lossFunc(pre_y, Y_val)
        
        
        # 점수 추출
        score= scoreFunc(pre_y, Y_val)
        

    # 에포크당 손실과 성능지표값 반환 
    # 테스트 및 검증 함수 사용
    return loss, score

In [ ]:
# 에포크 당 학습 진행 후 손실값과 성능지표값 반환 함수
def epochTraining(model, feature, target, lossFunc, optimizer, batch_count, is_regression=True, batch_size=32):
    loss_total, score_total=0, 0

    # 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    for batch in range(batch_count):
        # 데이터셋 추출 및 텐서화
        start=batch*batch_size
        end= start+batch_size

        X_train=torch.FloatTensor(feature[start:end].values).to(DEVICE)
        Y_train=torch.FloatTensor(target[start:end].values).reshape(-1,1).to(DEVICE)

        # 학습 진행
        pre_y=model(X_train)

        # 손실계산
        loss=lossFunc(pre_y, Y_train)
        loss_total+=loss
        
        # 점수 추출
        score= R2Score()(pre_y, Y_train) if is_regression else F1Score()(pre_y, Y_train)
        score_total+= score

        # 최적화 : weight, bias 업데이트
        optimizer.zerograd()
        loss.backward()
        optimizer.step()
    # 에포크당 손실과 성능지표값 반환 
    # 테스트 및 검증 함수 사용
    return loss_total/batch_count, 0

In [5]:
# 학습 진행
EPOCH=10
TV_loss={'Train':[], 'Val':[]}
for epoch in range(EPOCH):
    # 학습진행
    train_loss, train_score=epochTraining()
    # 검증진행
    val_loss, val_score=epochTesting()

    # 에포크당 학습 및 검증 결과 저장
    TV_loss['Train'].append(train_loss)
    TV_loss['Train'].append(train_score)

    TV_loss['Val'].append(val_loss)
    TV_loss['Val'].append(val_score)

    # 에포크당 학습 및 검증 결과 출력
    print(f'[{epoch}/{EPOCH}]\n[Train]\nloss: {train_loss}, Score: {train_score}\n\
[Val]\nloss: {val_loss}, score: {val_score}')

TypeError: epochTraining() missing 6 required positional arguments: 'model', 'feature', 'target', 'lossFunc', 'optimizer', and 'batch_count'